In [7]:
import json
import string
import pandas as pd
import re
import difflib
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime as dt
from nltk import ngrams
from langdetect import detect, DetectorFactory
from difflib import get_close_matches as gcm

In [8]:
skills = pd.read_csv('skills_db/skill.csv')
skills.head()

,skill_id,skill
0,150,Applied Science
1,163,Art
2,322,Business
3,548,Computer Science
4,714,Data Science


In [9]:
sk_list = skills['skill'].tolist()
len(sk_list)

3095

In [10]:
with open('resource/job_infos_scrape.json') as f:
    jobs = json.load(f)

len(jobs)

19680

In [11]:
jobs['0']

{'title': 'SALES ASSOCIATE',
 'job_details': '* Under supervision and perform duties to provide technical product assistance/knowledge to customers in order to generate new and repeat sales.\n* Assist customers in the selection of the best products that suited to their needs and desires and explain use, operation, care of the merchandise products and services to customers.\n* Explain the terms of sales, availability of the product and delivery dates and related information and process orders to customers.\n* Handle customers inquiries and complaint.\n* Keep display items clean and displayable and overall cleanliness of the branch.\n* To  handle  any  other  job  as  may be  assigned  by  the  superior.\n\nLocation : Tropicana Aman, Denai Alam, Wisma WCC, Puchong, Bandar Botanik, Sri Petaling, Pandan Indah, Rawang, Sungai Buloh, Kota Damansara, Kajang, Bangi , Balakong, USJ Taipan, Kota Kemuning, Shah Alam.',
 'requirement': 'Permanent| 3 Shift Time |STPM / A Level or Equivalent|Today',

In [12]:
rs = []
i = 0

for j in jobs:
    rs.append(jobs[j]['requirement'].split('|')[2])

set(rs)

{"Bachelor's or Equivalent",
 'Diploma / Advanced Diploma / Higher Graduate Diploma / DVM / DKM Level 4 / DLKM Level 5',
 'Diploma / Advanced Diploma / Higher Graduate Diploma / DVM /…',
 'Diploma / Advanced Diploma / Higher Graduate…',
 'Doctoral (PhD) or Equivalent',
 "Master's or Equivalent",
 'PMR / PT3 or Equivalent',
 'Primary Education or Below',
 'SPM / O Level / SKM Level 1 / SKM Level 2 / SKM Level 3 or Equivalent',
 'SPM / O Level / SKM Level 1 / SKM Level 2 / SKM Level 3 or…',
 'STPM / A Level or Equivalent'}

In [13]:
def extract_skills(info):
    words = list(set(re.sub('[\n|,|.|/|\(|\)]', ' ', info).lower().split()))
    bigrams = [' '.join(g) for g in ngrams(words, 2)]
    trigrams = [' '.join(g) for g in ngrams(words, 3)]
    results = []
    for skill in sk_list:
        s = skill.lower()
        if '(' in s:
            abb = s[s.find("(")+1:s.find(")")]
            if len(gcm(abb, words, cutoff=0.95)) > 0:
                results.append(skill)
                continue
            s = re.sub(r"[\(].*?[\)]", "", s)
        s2 = s.split()
        if len(s2) == 1:
            if len(gcm(s, words, cutoff=0.9)) > 0:
                results.append(skill)
        elif len(s2) == 2:
            if len(gcm(s, words, cutoff=0.85)) > 0:
                results.append(skill)
        elif len(s2) == 3:
            if len(gcm(s, trigrams, cutoff=0.8)) > 0:
                results.append(skill)
        else:
            if len(gcm(s, trigrams, cutoff=0.7)) > 0:
                results.append(skill)
    return results

In [15]:
i = 0
job_info_list = []
DetectorFactory.seed = 42
initial = dt.now()
interval = dt.now()
print_every = 500

for j in jobs:
    i += 1
    if i % print_every == 0:
        print("{} jobs processed. Time taken: {}".format(i, dt.now() - interval))
        interval = dt.now()
    job = jobs[j]
    all_info = job['title'] + ' ' + job['job_details']
    job_type = job['requirement'].split('|')[0]
    education = job['requirement'].split('|')[2]
    if len(job['job_details']) < 3:
        language = None
    else:
        language = detect(job['job_details'])
    skills = extract_skills(all_info)
    job_info_list.append({
        'job_title': job['title'],
        'job_details': job['job_details'],
        'job_type': job_type,
        'education_requirement': education,
        'language': language,
        'skills_required': skills,
        'no_skills_required': len(skills)})

print("Total time taken: {}".format(dt.now() - initial))

500 jobs processed. Time taken: 0:03:29.833196
1000 jobs processed. Time taken: 0:03:17.678279
1500 jobs processed. Time taken: 0:03:32.146844
2000 jobs processed. Time taken: 0:03:20.760168
2500 jobs processed. Time taken: 0:03:54.796798
3000 jobs processed. Time taken: 0:02:06.826951
3500 jobs processed. Time taken: 0:03:21.240380
4000 jobs processed. Time taken: 0:03:46.470786
4500 jobs processed. Time taken: 0:03:27.395169
5000 jobs processed. Time taken: 0:03:22.553503
5500 jobs processed. Time taken: 0:03:11.888057
6000 jobs processed. Time taken: 0:03:06.733814
6500 jobs processed. Time taken: 0:03:39.735435
7000 jobs processed. Time taken: 0:03:14.074949
7500 jobs processed. Time taken: 0:03:34.103895
8000 jobs processed. Time taken: 0:03:26.692215
8500 jobs processed. Time taken: 0:03:59.744204
9000 jobs processed. Time taken: 0:03:25.559526
9500 jobs processed. Time taken: 0:03:01.445648
10000 jobs processed. Time taken: 0:03:02.309910
10500 jobs processed. Time taken: 0:03:2

In [16]:
df = pd.DataFrame.from_dict(job_info_list)
df.head()

,job_title,job_details,job_type,education_requirement,language,skills_required,no_skills_required
0,SALES ASSOCIATE,* Under supervision and perform duties to prov...,Permanent,STPM / A Level or Equivalent,en,"[Sales, Operations, Process, Service, Selectio...",6
1,SIGN HOUSE ADVERTISING,Kelebihan diberi kepada yang mempunyai kemahir...,Permanent,Bachelor's or Equivalent,id,"[Adobe, Advertising]",2
2,MR DIY (M) SDN BHD,"Receive stock, checking & distribution item re...",Permanent,Diploma / Advanced Diploma / Higher Graduate…,en,"[Operations, Distribution]",2
3,Petugas PDK,"1. Bertanggungjawab kepada penyelia PDK, jawat...",Permanent,Bachelor's or Equivalent,id,[],0
4,MR DIY (M) SDN BHD,"Receive stock, checking & distribution item re...",Permanent,SPM / O Level / SKM Level 1 / SKM Level 2 / SK...,en,"[Operations, Distribution]",2


In [17]:
df.to_csv('resource/myfuturejobs_skills2.csv', index=False)